# Combined presentations and Proof of Association

Disclamer: This text is not intended to communicate a favored approach; but to guide a fruitful discussion. Furthermore, the text does not consider protocol choices already made, but may reference these when suitable.

## Defining combined presentations

[Topic K](https://github.com/eu-digital-identity-wallet/eudi-doc-architecture-and-reference-framework/issues/341) defines *combined presentations* and aims to outline implementation requirements.

A slight refinement could better clarify security requirements and shift the focus to what the Verifier must *determine* (rather than attribute requests and consolidation).

I propose defining a combined presentation as a transaction where at least two attributes from distinct attestations are presented to a Verifier so that the Verifier:

1. determines whether the two attributes describe the same identity subject (*Identity Binding*), and
2. ensures that the attributes are properly paired when presented (*Valid Composition*).

The first requirement is straightforward, but the second is more nuanced. **Example A** illustrates why proper pairing is crucial (see also the [VC implementation guidelines "selective disclosure" subsection under ZKP](https://www.w3.org/TR/vc-imp-guide/#zero-knowledge-proofs)).

> **Example A: The risk of improper attribute pairing**  
The following two attestations describe the same identity subject:
```
{
  "attestation_id": 1,
  "employer": "UK ltd",
  "role": "CEO",
  "name": "John Doe"
}
{
  "attestation_id": 2,
  "employer": "SE ABB",
  "role": "Board member"
}
```
A **valid composition** maintains logical consistency:
```
{
  "employer": "SE ABB",
  "name": "John Doe"
}
```
The following example is **NOT a valid composition** as it improperly pairs the `employer` and `role` values.
```
{
  "employer": "SE ABB",
  "role": "CEO"
}
```
## Security requirements for combined presentations

With the above in mind, the following security requirements must be met:

1. **Identity Binding**: The Verifier must determine whether the presented attributes refer to the same identity subject.
  * Note that this does not mean the attributes must describe the same identity subject—only that the Verifier can assess whether they do.
  * Some combined presentations may include attributes from different identity subjects (e.g., a child and their legal guardian).
2. **Valid Composition**: The Verifier must ensure that all attributes are properly paired according to their original attestations and **not** combined in a way that alters their intended meaning.

## Achieving Identity Binding

To determine Identity Binding, there needs to be some sort of Proof of Association between the presented attributes. I am aware of the following general solutions:

### Session-based PoA

Session-based PoA ensures that all attestations sent within a secured session originate from the same certified WSCD and refer to the same identity subject. This approach is useful when a Verifier can validate multiple attestations within a session.

Session-based PoA requires that:

1. The WSCD must protect the User's attestations, not just the cryptographic keys.
2. The WSCD must only protect attestations that describe the same identity subject.
3. The Verifier must bind the secure session to a User identity and WSCD.

I am not familiar enough with session-based PoA approaches to provide a detailed list of drawbacks and benefits, but some considerations may include:

* **Questionable offline support**. Given that a session must exist for PoA, offline support is difficult.
* **Lightweight**. No need for cryptography beyond establishing a secure session as all attestations sent over the session are assumed to be associated.
* **Adaptable to existing protocols**. Existing standards like OAuth 2.0, OIDC, and TLS session binding can be leveraged but I am unaware of a widely deployed session-based PoA-specific standard.

### Claim-based PoA

Claim-based binding allows a Verifier to perform identity binding solely based on the claims presented by the User. There are two general approaches:

1. Unblinded PoA inputs (Verifier linkable)
2. Blinded PoA inputs (Verifier unlinkable)

Since Verifier unlinkability is a requirement, we will only consider the Verifier unlinkable blinded PoA inputs below.

> Note: Issuer unlinkability is irrelevant if the attestation includes other Issuer-linkable correlation handles (e.g., salt values, public keys etc.).

To achieve Verifier unlinkability, the User only present blinded claim statements. Blinded claim statements are particularly suitable when attestation Issuers rely on a attestation (e.g., a PID) to identify the User before issuing new attestations. In this model, Issuers and Verifiers receive different identifying attestations:

* Issuers recieve a PID that contains a claim statement with a unique value (only shown to Issuers). Each Issuer blinds the claim statement before embedding it in the issued attestation.
* The User reveals only blinded claim statements with a PoA to a Verifier.

During verification, the User must prove that the blinded claim statements are equivalent in some way (c.f. Appendix A for a ZKP-based approach that offers third party repudiation).

Some limitations of blinded claim-based PoAs include:

* **Verifier burden**. Verifiers must process and validate the PoA.
* **Issuer burden**: Issuers must create and include blinded PoA inputs in each attestation.
* **Not standardized**. A blinded claims based PoA requires standardization and also protocol support.
* **Single-use**. Blinded claim values are static unique values and thus single-use only to prevent Verifier linkability.

Benefits include:

* **Compliant with existing protocols**. A blinded claim based PoA is entirely software based and can be achieved using a separate PoA endpoint / out-of-band channel.
* **Separation of concerns**. The PoA does not require WSCD protection. However, a Verifier will only request a PoA if the User proves PoP of WSCD-protected keys in the Combined Presentation.
* **Repudiable compositions**. With an IZKP (but not NIZKP) PoA, a Verifier can generate a simulated transcript indistinguishable from a real one, providing Users repudiability against third parties.
* **Curve\algo independence**. With a ZKP-based PoA, you can prove that two keys are associated even across curves and algorithms (e.g., signature and key exchange).

### Signature-based PoA  

Signature-based PoA uses at least two keypairs in a process that establishes their association. These approaches **do not inherently provide PoA** and instead rely on certified hardware to enforce signing rules. Possible adaptations could leverage:
  * **Countersignatures**: One key signs a signature from another key, ensuring both are linked, e.g., $Sign(k_a, Sign(k_b, M))$.
  * **Cross-signing**: A private key signs a statement containing another public key, proving their relationship, e.g., $Sign(k_a, K_b)$.
  * **Multi-signatures**: Multiple keys jointly create a valid signature. Some schemes, like Schnorr, allow for signature aggregation, e.g., $Sign(k_a, k_b, M)$.

There are some major limitation of signature-based PoA.

* **Requires certified hardware**. Every signature-based PoA approach relies on certified hardware to enforce signature policies that only allow the generation of PoAs if all involved keys are under the same device's sole control. Without it, an attacker can trivially generate PoAs.
* **Repurposed techniques**. A signature-based PoA repurposes cryptographic methods that were not designed to inherently enforce PoA and must be adapted for this purpose:
  * Countersignatures are used for stepwise accountability, such as timestamping, document notarization, and email receipt non-repudiation.
  * Cross-signing is intended for a trusted entity vouching for another trusted entity's key. It can be used for identity binding; implementations of PGP rely on cross-signing to establish identity binding between a primary key and a subkey.
  * Multi-signatures are used for threshold security and collaborative signing.
* **Verifier burden**. The Verifier must process and validate the PoA. While not equally complex, signature based PoAs add overhead and requires a peer reviewed standardization process.
* **Linkability**. Signature-based PoA publicly links two keys, making it suitable only for one-time-use keys.
* **Non-repudiable compositions**. Signature-based PoA creates a permanent, non-repudiable link between two attestations, exceeding the minimal proof required and enabling third-party behavioral profiling.

Despite its drawbacks, signature-based PoA offers several benefits:

* The cryptographic primitives are widely understood and easy to implement.
* Signature-based PoA techniques are highly unlikely to raise patent concerns.
* While these techniques require repurposing, they are well-documented, widely used, and supported by clear standards.


### Related key PoA

A related key PoA establishes an association between public keys based on relationships between their private keys.

> NOTE: Related key PoA has known security risks (e.g., with ECDSA) and potential patent concerns, requiring careful and extensive examination.

Two primary approaches exist. Both leverage homomorphic relationships between private and public key operations:

1. **Key ratio associations**. [Verheul (2025)](https://eprint.iacr.org/2024/1444.pdf) presents a well-developed approach.
  * An **association key** is defined as the ratio of private key scalars: $z = p_2 \cdot p_1^{-1}$.
  * The multiplicative structure of $z$ is preserved in the public key domain enabling PoA.
  * Based on a modified Schnorr signature.
  * Requires both PoA and PoP from the User.
  * Depends on certified WSCD capable of arbitrary private key computations.
2. **Derived associations with key blinding**. Detailed in BIP-32 and SLIP-0010 (both widely adopted), adapted for an EUDIW context in [ARKG](https://datatracker.ietf.org/doc/draft-bradleylundberg-cfrg-arkg/), and [HDK](https://datatracker.ietf.org/doc/draft-dijkhuis-cfrg-hdkeys/); further detailed in [KBSS](https://datatracker.ietf.org/doc/draft-irtf-cfrg-signature-key-blinding/).
  * Applies a blinding factor, $p_b$, to an existing private key $p_1$.
  * The blinding factor can be applied additively, $p_1 + p_b$, or multiplicative $p_1 \cdot p_b$, depending on the scheme and leverages existing key generation subroutines.
  * When locally computed the User derives $p_b$ for use with new keys. An Issuer would here require PoP and PoA.
  * In joint computation, a KEM ensures that both the User and Issuer can compute the public key association, while only the User derives the private key association.
  * Implicitly ensures PoP and PoA requiring proof of neither.
  * Explicit PoA possible via Chaum-Pedersen proofs.

> NOTE 1: Key generation and PoA are intertwined in key ratio assocations, but separate (and potentially software based) in derived assocation using key blinding.
> NOTE 2: Due to the relationships introduced, both approaches require safeguards with malleable schemes like ECDSA to prevent signature forgeries.

Both approaches require extensive scrutiny before their benefits and drawbacks can be fully assessed. This discussion aims to contribute to their respective evaluations.

As I actively support and contribute to the derived associations via key blinding approach, I would like to initiate this discussion with the following remarks:
* Related discussion: See [Topic B](https://github.com/eu-digital-identity-wallet/eudi-doc-architecture-and-reference-framework/discussions/354#discussioncomment-11941084)
* **Critical** review of Verheul (2015): Available in this [publicly accessible document](https://drive.google.com/file/d/1zBzYR4ZM5gxt_IdNDOMuaxlztakyjAcv/view?usp=sharing), where all comments are public.

I greatly appreciate Verheul's work and the effort invested; my critique is intended to be constructive and support further improvement. To this end, I offer the following comments:

* The security requirements are heuristically derived from a specific use case context that may not align with EUDIW's needs.
* Particularly, I challenge SW2 in Verheul's work, but also raise issues with SW1.
* Seemingly, due to the single-generator constraint and potentially overly restrictive security assumptions, Verheul's approach introduces three key requirements:
  * (a) an association key,
  * (b) a WSCD capable of arbitrary private key operations, and
  * (c) a heavily modified Schnorr algorithm.
* I am concerned about the relationship introduced in (a) and that (c) deviates significantly from a standard Schnorr or Chaum-Pedersen proof.
* If (b) is required, it raises the question of why (a) and (c) are needed at all, given that simpler methods (c.f. Appendix B) may exist.
* Verheul's approach blurs the distinction between Proof of Possession (PoP) and Proof of Association (PoA), which may not be ideal for modular security designs.
* The HDK approach I am involved in has received criticism as discussed in Topic B.

## Achieving Valid Composition

I have no idea how to do this generally. The only viable ways I can think of are:

1. limit attribute compositions to a single identity subject, and
2. a claims based approach where attributes evidence valid compositions (e.g., values in the first attestation for `parents` that can be matched with the corresponding subject identifer values in the second attestation).

# Appendices

## Appendix A: A third party repudiable ZKP-based PoA

In [1]:
%%capture
!pip install ecpy
import secrets
from ecpy.curves    import Curve, Point

An EC setup requires a curve and two generators, $(G, H)$. $G$, is the curve's standard generator, while security requires $H$ to be chosen using a publicly verifiable random process. We need to ensure each Issuer generates a distinct generator and eliminate as many attack vectors as possible.

One potential solution is a KEM-seeded Hash-to-Curve ([RFC 9180](https://www.rfc-editor.org/rfc/rfc9180.html) with [RFC 9380](https://www.rfc-editor.org/rfc/rfc9380.html)). The KEM part ensures Issuer distinct generators, while Hash-to-Curve outputs a generator with unknown scalar.

Here, we just pick $H$ at random.

In [2]:
# The curves
crv = Curve.get_curve("secp256r1")
order = crv.order
G  = crv.generator

# 1st blind generator
H_1 = secrets.randbelow(order) * G

# 2nd blind generator
H_2 = secrets.randbelow(order) * G

The PID issuer includes a unique random value, `poa_seed`, in the PID attestation. Note that this value is **NOT** intended to act as a unique identifier for the User, but as a PoA input.

In [3]:
# The PID issuer includes a PoA seed in the attestation
poa_seed = secrets.randbelow(order)

In the next step, two additional distinct Issuers rely on the poa_seed to generate a blinded version each, $H_1, H_2$. Note that only the User sees both blinded values at this step.

In [4]:
# Attestation Issuer 1
blind_poa_input_1 = poa_seed * H_1

#Attestation Issuer 2
blind_poa_input_2 = poa_seed * H_2

The User can now reveal the two blinded PoA values to a Verifier. The Verifier will require the User to generate a PoA between $A$ and $B$.

The User can generate a PoA proof using e.g., a PODLE like the Chaum-Pedersen protocol, which allows the User to prove that it knows two discrete logarithms $y_1 = h_1^r$ and $y_2 = h_2^r$ with respect to two distinct generators $h_1, h_2$.

The Verifier accepts the proof of discrete log equivalence as a proof of association.

In [5]:
# User samples a random nonce
r = secrets.randbelow(order)

# The User shares two commitments with the Verifier
P = r * H_1
Q = r * H_2

# A random challenge is produced either by the Verifier or using a random oracle.
# Here we just sample from random
# A repudiable proof requires Verifier picked c thus IZKP
# A NIZKP with Fiat-Shamir is always third party non-repudiable
c = secrets.randbits(256)

# The user computes s
s = r + c * poa_seed

# The Verifier verifies the PODLE using challenge c, recieved values (s, P, Q), and blinded PoAs
assert (s * H_1 == P + (blind_poa_input_1 * c))
assert (s * H_2 == Q + (blind_poa_input_2 * c))

The ZKP property means that there exists a simulator that can produce proofs indistinguishable from real ones even without knowledge of the secret. This is useful when a Proof of Association (PoA) needs to be repudiable. Works only with IZKP and not NIZKP (the latter does not permit the challenge to be selected before R).

The **real proof**, given $Y = x \cdot H_1$ and $Z = x \cdot H_2$, where the Prover knows $x$ consists of:

* Prover commitments: $P = r \cdot H_1, Q = r \cdot H_2$
* A random challenge $c$
* Prover computation: $s = r + c \cdot x $

The **simulated proof** is:

* Pick $s$ at random
* Compute $P = s \cdot H_1 - c \cdot Y, Q = s \cdot H_2 - c \cdot Z$

The transcript is verified with:

* $s \cdot H_1 = P + c \cdot Y$
* $s \cdot H_2 = Q + c \cdot Z$

Since the simulated transcript is computationally indistinguishable from a real one, a third party cannot determine whether the Prover actually participated. This enables repudiability of the PoA, meaning the Prover can plausibly deny involvement in the proof.

## Apprendix B: Simple PoA when WSCD supports arbitrary operations

Alg 1-2 in Verheul presents a way to generate a PoA between two private keys protected by the same WSCD. Verheul's proposal builds on the assumption that the Prover "has full control over both private keys" and can "do arbitrary mathematical operations with these." This is seemingly required because of the limitation to rely on a the curve specified generator $G$ only.

One possible argument raised against Alg 1-2 is that they are overly complex given that the assumption is *full* control. A WSCD that has full control can simplify the PoA.

In [6]:
# Using the same notation as Alg 1-2
p_1 = secrets.randbelow(order)
p_2 = secrets.randbelow(order)

# The two public keys the Prover wants to generate a PoA for
P_1 = p_1 * G
P_2 = p_2 * G

# Instead of an association key, the WSCD outputs a third value z
z = p_1 + p_2

# The verification is now
P_1 + P_2 == z*G

True

The above approach is extendable to more than two private keys. Given the scalar sum $z = \sum_{i=0}^n{p_i}$, the corresponding public key sum is: $\sum_{i=0}^n{P_i} = z \cdot G$, which serves as a combined PoP and PoA.

Since each private key is a random value, their sum is indistinguishable from a randomly chosen scalar from the same group. Knowledge of $z$ does not leak information about either $p_1$ or $p_2$ without additional information; it is information theoretically secure.

An attacker with knowlege of $(P_1, P_2, z)$ cannot extract either $p_1$ or $p_2$ from $(P_1, P_2)$ because that would break ECDLP.

Seemingly, an exhaustive search is the best approach for the attacker in the case of single-use-keys, which is the same as the attacker has for guessing either $p_1$ or $p_2$ from $P_1$ or $P_2$.